In [1]:
import torch
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModel, RobertaTokenizerFast, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [100]:
dataset = load_dataset("carblacac/twitter-sentiment-analysis", "None");

dataset['train'][10]

Using custom data configuration None
Found cached dataset twitter-sentiment-analysis (/Users/oliverklingefjord/.cache/huggingface/datasets/carblacac___twitter-sentiment-analysis/None/1.0.0/cd65e23e456de6a4f7264e305380b0ffe804d6f5bfd361c0ec0f68d8d1fab95b)



100%|██████████| 3/3 [00:00<00:00, 77.71it/s]


{'text': 'About to go to bed. Sleeping really late tomorrow!  I am so glad the Tigers won tonight!!',
 'feeling': 1}

In [5]:
MODEL = "cardiffnlp/twitter-roberta-base-2021-124m"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2)

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

OSError: We couldn't connect to 'https://huggingface.co' to load this model, couldn't find it in the cached files and it looks like cardiffnlp/twitter-roberta-base-2021-124m is not the path to a directory containing a config.json file.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

# Set up the tokenizer function

In [ ]:
def tokenizer_fn(data):
    return tokenizer(data['text'], truncation=True, padding="max_length", return_tensors='pt')

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'feeling'],
        num_rows: 119988
    })
    validation: Dataset({
        features: ['text', 'feeling'],
        num_rows: 29997
    })
    test: Dataset({
        features: ['text', 'feeling'],
        num_rows: 61998
    })
})

In [ ]:
dataset = dataset.map(tokenizer_fn, batched=True)

Loading cached processed dataset at /Users/oliverklingefjord/.cache/huggingface/datasets/carblacac___twitter-sentiment-analysis/None/1.0.0/cd65e23e456de6a4f7264e305380b0ffe804d6f5bfd361c0ec0f68d8d1fab95b/cache-5ba45a1d014f30a6.arrow





































































  0%|          | 0/189 [16:36:39<?, ?it/s]





















 97%|█████████▋| 29/30 [00:06<00:00,  4.40ba/s]
Loading cached processed dataset at /Users/oliverklingefjord/.cache/huggingface/datasets/carblacac___twitter-sentiment-analysis/None/1.0.0/cd65e23e456de6a4f7264e305380b0ffe804d6f5bfd361c0ec0f68d8d1fab95b/cache-51f87f6b59f11e97.arrow


In [ ]:
small_train_dataset = dataset['train'].shuffle(seed=42).select(range(1000)).rename_column("feeling", "labels").remove_columns('text')
small_validation_dataset = dataset['validation'].shuffle(seed=42).select(range(1000)).rename_column("feeling", "labels").remove_columns('text')

Loading cached shuffled indices for dataset at /Users/oliverklingefjord/.cache/huggingface/datasets/carblacac___twitter-sentiment-analysis/None/1.0.0/cd65e23e456de6a4f7264e305380b0ffe804d6f5bfd361c0ec0f68d8d1fab95b/cache-5a1aca06fe9bdaf3.arrow
Loading cached shuffled indices for dataset at /Users/oliverklingefjord/.cache/huggingface/datasets/carblacac___twitter-sentiment-analysis/None/1.0.0/cd65e23e456de6a4f7264e305380b0ffe804d6f5bfd361c0ec0f68d8d1fab95b/cache-ea60b86d0cd4afad.arrow


In [ ]:
training_args = TrainingArguments(
    output_dir="test_trainer", 
    evaluation_strategy="epoch", 
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
)
metric = load_metric("accuracy")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, -1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_validation_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/Users/oliverklingefjord/.pyenv/versions/3.10.7/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 189
  0%|          | 0/189 [00:00<?, ?it/s]